# Model Evaluation

### First need to set up Model

Run the code that sets up model

In [116]:
import pandas as pd

#import GPUtil

import argparse
import json
import logging
import os

import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
#import torchvision
#import torchvision.models
#import torchvision.transforms as transforms

from typing import List
import numpy as np

## Set up logger to get details of errors
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [117]:
# k is for number of nodes in each hidden layer of NN
k = 1000

# For number of inputs (32 binary digits)
input_length = 32
output_length = input_length

# Model Parameters
epochs = 1
batch_size = 256
lr = 0.001
momentum = 0.9

device = "cuda" if torch.cuda.is_available() else "cpu"

In [118]:
class Generator(nn.Module):
    def __init__(self, output_length: int):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(output_length, k)
        self.dense_layer2 = nn.Linear(k, k)
        self.dense_layer3 = nn.Linear(k, k)
        self.dense_layer4 = nn.Linear(k, output_length)

    def forward(self, x):
        l1 = self.dense_layer(x)
        l2 = self.dense_layer2(F.relu(l1))
        l3 = self.dense_layer3(F.relu(l2))
        l4 = self.dense_layer4(F.relu(l3))
        return F.sigmoid(l4)

In [119]:
generator = Generator(output_length)
generator = generator.to(device)

### Load in Trained Generator

In [108]:
generator.load_state_dict(torch.load('gen9.pt')["generator_state_dict"])

<All keys matched successfully>

In [109]:
noise = torch.randint(0, 2, size=(batch_size, output_length)).float().to(device)

In [110]:
def extract(G_of_noise):

    G_numpy = G_of_noise.detach()   

    curves = []

    for i in range(len(G_numpy)):
        c1 = int(G_numpy[i][0].round())
        c2 = ((-1)**(int(G_numpy[i][1].round())))*(int(G_numpy[i][2].round()))
        c3 = int(G_numpy[i][3].round())
        c4 = (-1)**(int(G_numpy[i][4].round()))*(int("".join([str(int(y)) for y in G_numpy[i][5:18].round()]), 2))
        c6 = (-1)**(int(G_numpy[i][12].round()))*(int("".join([str(int(y)) for y in G_numpy[i][19:].round()]), 2))
    
        coef = [c1,c2,c3,c4,c6]
        curves.append(coef)
    return curves

### Check Results

In [114]:
extract(generator(noise))

/local/scratch/jcox22/anaconda3/envs/pytorch110/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[[0, 0, 1, -3035, 6549],
 [0, 0, 1, -3657, 3229],
 [0, 0, 1, 2655, 155],
 [0, -1, 1, -2651, 2203],
 [0, 0, 1, 2429, -2195],
 [0, 0, 1, -2905, 6301],
 [0, 0, 1, -2911, 2207],
 [0, 0, 0, -2393, 2453],
 [0, 0, 1, 2907, 147],
 [0, -1, 1, 7007, 2205],
 [0, 0, 1, 6479, 159],
 [0, 0, 1, 2909, 3479],
 [0, 0, 0, -2639, 6301],
 [0, 0, 1, -2393, 3477],
 [0, 0, 1, -2907, 153],
 [0, -1, 1, -2891, 2201],
 [0, 0, 0, -2143, 4249],
 [0, 0, 1, 3679, 2205],
 [0, 0, 1, 2935, -1047],
 [0, 0, 1, -3675, 156],
 [0, -1, 1, -2523, 6551],
 [0, 0, 1, -2619, -151],
 [0, -1, 0, -2909, 6295],
 [0, 0, 0, 2431, -6299],
 [0, -1, 0, 2943, -3229],
 [0, 0, 1, -2907, 4249],
 [0, -1, 1, -2927, -6297],
 [0, 0, 1, -2393, 2197],
 [0, 0, 1, 3931, 6161],
 [0, 0, 0, -3931, 2205],
 [0, -1, 0, 6985, 5277],
 [0, 0, 1, -2911, 6301],
 [0, -1, 1, 2907, 2461],
 [0, 0, 1, 2427, -155],
 [0, 0, 0, -2911, 157],
 [0, 0, 1, 3929, 4247],
 [0, 0, 1, 3547, 6553],
 [0, 0, 0, 2685, -2201],
 [0, 0, 1, -2397, 6173],
 [0, 0, 0, 3935, 6293],
 [0, -1, 

### Check to see how many are u

In [111]:
curves = pd.DataFrame(extract(generator(noise)))

/local/scratch/jcox22/anaconda3/envs/pytorch110/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [112]:
curves

,0,1,2,3,4
0,0,0,1,-3035,6549
1,0,0,1,-3657,3229
2,0,0,1,2655,155
3,0,-1,1,-2651,2203
4,0,0,1,2429,-2195
...,...,...,...,...,...
251,0,0,0,-2905,159
252,0,0,1,-2909,2201
253,0,0,1,2943,-2193
254,0,0,1,2891,159


In [113]:
curves.drop_duplicates()

,0,1,2,3,4
0,0,0,1,-3035,6549
1,0,0,1,-3657,3229
2,0,0,1,2655,155
3,0,-1,1,-2651,2203
4,0,0,1,2429,-2195
...,...,...,...,...,...
251,0,0,0,-2905,159
252,0,0,1,-2909,2201
253,0,0,1,2943,-2193
254,0,0,1,2891,159
